<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/03_pytorch_computer_vision_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. PyTorch Computer Vision Exercises

The following is a collection of exercises based on computer vision fundamentals in PyTorch.

They're a bunch of fun.

You're going to get to write plenty of code!

## Resources

1. These exercises are based on [notebook 03 of the Learn PyTorch for Deep Learning course](https://www.learnpytorch.io/03_pytorch_computer_vision/). 
2. See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/_PibmqpEyhA). 
  * **Note:** Going through these exercises took me just over 3 hours of solid coding, so you should expect around the same.
3. See [other solutions on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions).

In [1]:
# Check for GPU
!nvidia-smi

Wed Jan 24 10:40:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8               3W /  35W |     71MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# Import torch
import torch

# Exercises require PyTorch > 1.10.0
print(torch.__version__)

# TODO: Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device


2.1.2+cu118


'cuda'

## 5. Load the [`torchvision.datasets.MNIST()`](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST) train and test datasets.

In [2]:
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor

PATH = 'D:\\Coding_Stuff\\GitHub\\ZTM-PyTorch\\datasets\\FasionMNIST\\FashionMNIST\\raw'
train_data = MNIST(root=PATH,
                   download=True,
                   train=True,
                   transform=ToTensor())
test_data = MNIST(root=PATH,
                  download=True,
                  train=False,
                  transform=ToTensor())

100%|██████████| 9912422/9912422 [00:01<00:00, 5955980.68it/s]


Extracting D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw\train-images-idx3-ubyte.gz to D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]

Extracting D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw\train-labels-idx1-ubyte.gz to D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 8262990.16it/s]


Extracting D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw\t10k-images-idx3-ubyte.gz to D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw\t10k-labels-idx1-ubyte.gz to D:\Coding_Stuff\GitHub\ZTM-PyTorch\datasets\FasionMNIST\FashionMNIST\raw\MNIST\raw



## 6. Visualize at least 5 different samples of the MNIST training dataset.

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data,
                              shuffle=True,
                              batch_size=32,)
test_dataloader = DataLoader(test_data,
                             shuffle=True,
                             batch_size=32)


## 7. Turn the MNIST train and test datasets into dataloaders using `torch.utils.data.DataLoader`, set the `batch_size=32`.

## 8. Recreate `model_2` used in notebook 03 (the same model from the [CNN Explainer website](https://poloclub.github.io/cnn-explainer/), also known as TinyVGG) capable of fitting on the MNIST dataset.

## 9. Train the model you built in exercise 8. for 5 epochs on CPU and GPU and see how long it takes on each.

## 10. Make predictions using your trained model and visualize at least 5 of them comparing the prediciton to the target label.

## 11. Plot a confusion matrix comparing your model's predictions to the truth labels.

## 12. Create a random tensor of shape `[1, 3, 64, 64]` and pass it through a `nn.Conv2d()` layer with various hyperparameter settings (these can be any settings you choose), what do you notice if the `kernel_size` parameter goes up and down?

## 13. Use a model similar to the trained `model_2` from notebook 03 to make predictions on the test [`torchvision.datasets.FashionMNIST`](https://pytorch.org/vision/main/generated/torchvision.datasets.FashionMNIST.html) dataset. 
* Then plot some predictions where the model was wrong alongside what the label of the image should've been. 
* After visualing these predictions do you think it's more of a modelling error or a data error? 
* As in, could the model do better or are the labels of the data too close to each other (e.g. a "Shirt" label is too close to "T-shirt/top")?